In [ ]:
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils
!pip install pytesseract pdf2image neo4j

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
pip install pdf2image

In [ ]:
pip install pytesseract

In [ ]:
import requests
import pdf2image
import pytesseract

pdf = requests.get('https://arxiv.org/pdf/2110.03526.pdf')
doc = pdf2image.convert_from_bytes(pdf.content)

# Get the article text
article = []
for page_number, page_data in enumerate(doc):
    txt = pytesseract.image_to_string(page_data).encode("utf-8")
    # Sixth page are only references
    if page_number < 6:
      article.append(txt.decode("utf-8"))
article_txt = " ".join(article)


# Text preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
  """Remove section titles and figure descriptions from text"""
  clean = "\n".join([row for row in text.split("\n") if (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                    and not row.startswith("Figure")])
  return clean

text = article_txt.split("INTRODUCTION")[1]
ctext = clean_text(text)
sentences = nltk.tokenize.sent_tokenize(ctext)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
pip install retrying

In [ ]:
import hashlib
import requests
from retrying import retry  # Import retry from the retrying library

# Decorator for retrying the query_raw function
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_attempt_number=5)
def query_raw(text, url="http://bern2.korea.ac.kr/plain", timeout=10):
    try:
        response = requests.post(url, json={'text': text}, timeout=timeout)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        return None

# Update the code with the modified query_raw function
entity_list = []
# The last sentence is invalid
for s in sentences[:-1]:
    entity_list.append(query_raw(s))

parsed_entities = []
for entities in entity_list:
    e = []
    # If there are no denotations in the text
    if not entities.get('denotations'):
        parsed_entities.append({'text': entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
        continue
    for entity in entities['denotations']:
        other_ids = [id for id in entity['id'] if not id.startswith("BERN")]
        entity_type = entity['obj']
        entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
        try:
            entity_id = [id for id in entity['id'] if id.startswith("BERN")][0]
        except IndexError:
            entity_id = entity_name
        e.append({'entity_id': entity_id, 'other_ids': other_ids, 'entity_type': entity_type, 'entity': entity_name})
    parsed_entities.append({'entities': e, 'text': entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})

# Printing the parsed entities for demonstration
for parsed_entity in parsed_entities:
    print(parsed_entity)


Error occurred: HTTPConnectionPool(host='bern2.korea.ac.kr', port=80): Read timed out. (read timeout=10)
Error occurred: HTTPConnectionPool(host='bern2.korea.ac.kr', port=80): Read timed out. (read timeout=10)
Error occurred: HTTPConnectionPool(host='bern2.korea.ac.kr', port=80): Read timed out. (read timeout=10)
Error occurred: HTTPConnectionPool(host='bern2.korea.ac.kr', port=80): Read timed out. (read timeout=10)
Error occurred: HTTPConnectionPool(host='bern2.korea.ac.kr', port=80): Read timed out. (read timeout=10)


KeyboardInterrupt: ignored

In [ ]:
parsed_entities=[
    {'text': 'Many people with skin diseases such as chronic wounds, non-healing and diabetic ulcers need reconstruction and regeneration of their skin.', 'text_sha256': '95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6cbfa373f176c9d39ae'},
{'text': 'In addition, the medical industry also needed a method of skin rejuvenation and reconstruction for cosmetic purposes, even for healthy people.', 'text_sha256': '3424add78e6e71cf20906f1fc0bd050ace5cd3d9f71a430c2f8535dd21354f66'},
{'text': 'Reconstructive medicine used the method to deliver pluripotent stem cells to the 33 years after the introduction of bone marrow stem cells, fat-derived stem cells have become an excellent source for cell therapy.', 'text_sha256': 'b0c0e0caafe6a371790c385375c5ab9a5edfd695d152ccb5137afdf545e190ab'},
{'text': 'In 1961, two Canadian scientists first introduced stem cells.', 'text_sha256': 'c3ca44e1a8c1fde494adee19104303a349bec064702c50f7dd4283c02aae65f5'},
{'text': 'These cells, later found to be hematopoietic stem cells, have been used successfully to treat leukemia and some severe autoimmune diseases called bone marrow transplants.', 'text_sha256': '9785b08aa0fecd2e0a28784a7f550f08201c5fafed3e2fd72cb3e3c4c0faf19a'},
{'text': 'In 1968, another stem cell was introduced into the bone marrow, which has been shown to be effective due to its high ability to regulate immunity in many diseases, including skin, bone, joint diseases, heart, brain, nerves, and kidney.', 'text_sha256': 'a5a27a6b13d60968783b2e161c2f06b631e4f2efb9c9b03397c7835840d13b89'},
{'text': 'Nevertheless, it took scientists another 33 years to discover that human adipose tissue also has a stem cell, similar to bone marrow stem cells.', 'text_sha256': '028cec8f7cf00063675b02869b6c424d923e2f0661c30d9331c4942007b01c21'},
{'text': 'Each gram of human fat has 100 to 500 times more stem cells than bone marrow and is easier to obtain.', 'text_sha256': '99c395a72666cd8d0423c79e1d5f42daeda06b03d80e76a36ba991024c84408c'},
{'text': 'Although there are differences in bone marrow stem cells and fat, these advantages have attracted the attention of many researchers.', 'text_sha256': 'a2c71a4c77b7e9ac0c40a8c5fce3b144f2a38c7c24864c3f84874d4dc47c5d9a'},
{'text': 'Mesenchymal stem cells are the most interesting because when they get placed in the right environment and are coupled with the proper growth factors, they can choose a pathway to differentiate into that targeted tissue.', 'text_sha256': '14158336512ad4f512d796f95bed14dab9902aa06c9b2d352216bb51fb7e5df8'},
{'text': 'Also, they are very accessible, inexpensive, easy to extract, and reproducible.', 'text_sha256': 'd4dbd1fb9e7e5285e6546710fe83bf7ea44a809fe45c245e25e634c5d33f8653'},
{'text': 'The sources of these mesenchymal stem cells are bone marrow, bone, connective tissues, fats, etc.', 'text_sha256': 'a41788937f1559e1784948e625706fb4b42ca9e5f78de34184722993878af620'},
{'text': 'The best reservoir that has recently been discovered for mesenchymal cells is fat.', 'text_sha256': '0d4c5e19ee53e9318756e74cf2aeb8d9300d44f0ea8d6a7a01e62ed5b1d908f0'},
{'text': 'Fats are made out of fibrous, collagen fibers, and fibroblasts.', 'text_sha256': '61cd48281cbbbe54f962eac9251efd58747bbd63e4f53d148a1e9fb38584b334'},
{'text': 'On one hand, by injecting collagenase enzymes into the fat, all the collagen fibers and fibroblasts would slip away, and the pure mesenchymal stem cells would be available.', 'text_sha256': '50d0f4e2ae5e02d9618325ae8cc864b9208eaf9e34c3ac944ba073e058ea1633'},
{'text': 'On the other hand, In the engineered stem cell vascular fraction (SVF), which is a pellet containing mesenchymal cells, growth factors, inflammatory cells like macrophage-monocyte and fibroblasts, the mesenchymal cells get separated from all the other components by the centrifugation process.', 'text_sha256': '51c4910f2b4f40b8dd2a79181a734844cfc89d16e67ef6badf378470e73cf3de'},
{'text': 'Placing the mesenchymal stem cells into the targeted environment and providing the proper growth factors can produce fibroblasts and collagen to reconstruct the skin.', 'text_sha256': '23b6860ad70b95fc3013fbd75eb2165bba1a0ccab1bb166c9bc71e41be803871'},
{'text': 'As mentioned above, the stem cell environment is critical, and there is a possibility of stem cell movement due to the blood flow and paracrine, which the stem cell would secrete and affect its environment.', 'text_sha256': '43b81e871c1f1a66d0646735cc81d330c1ab19a013ba58cd3f3281f3246b1f07'},
{'text': 'Therefore, the matrix that keeps the stem cell in its place from getting flushed away by the blood flow and affecting its environment, known as the scaffold, was discovered.', 'text_sha256': '7cff074d2e8a691cf68d11d134c4f4eb04153c0034338700f784236e969a3952'},
{'text': 'The scaffold has to have specific characteristics that would avoid a foreign body reaction; It could not be antigenic ,allergic, or infectious.', 'text_sha256': '3b7b8620ed5944b531c7700750151aed83c98aecf3bac9bf454198dc36bf0f7b'},
{'text': 'Additionally, it needed to be accessible and cheap to produce.', 'text_sha256': '3cc8fc706ee9b1439fb08ac70c6b895ee0779a02642d0bf123acabdc1658282c'},
{'text': "The newest method utilized has been using alginate scaffolds, containing the mesenchymal cells and the essential growth factors, into the targeted cell resulting in stimulating the CD44 receptors on the skin's fibroblasts, causing a rush of collagen production.", 'text_sha256': 'b38bda76c13cd4a324995aa3e69f225c22991c21d14ca234b04927d75563e681'},
{'text': 'As we learned in our lectures, the green fluorescent protein bone marrow was implanted into non-green fluorescent protein mice resulting from HCT, (hematopoietic cell transplantation) improved the animal survival in a mouse model, which was surprising.', 'text_sha256': 'b9905fc5a29947f9f2b128858bb4773f599f9aacc17e9253b9b755b1891f6007'},
{'text': 'In this paper, the tissue regeneration of skin and the probability of hair growth from stem cells are being investigated.', 'text_sha256': 'f73782c49357a03073a9320fab4c957858e624cd04c18034038eb4a3031ac9bb'},
{'text': 'Over time, The biological skin replacement has dramatically improved from individual application of skin cells to cells and scaffold combinations for treatments, healing, and acute and chronic skin wounds.', 'text_sha256': 'c312653b440c5fa471cbb7593bd437e436761221b79de9d1257a7ff19f6afff0'},
{'text': 'Permanent skin substitutes with cellular engineering combination has shown to have crucial wound closure of more than 90% of the total body surface region in burns [7].', 'text_sha256': 'a864077dd9edf134a7d4cab957d410ef97b32e0020aa2fc6c90fddbe1d00272b'},
{'text': 'As an advantage the morbidity and mortality of acute and chronic wounds have brought lower; However, yet they have been unable to eliminate all skin mechanisms and functions [7].', 'text_sha256': '226bdde36a8bba656d943f5c78f441535ccf382ffc8b579db4874b25597c29d0'},
{'text': 'Therefore, Hypopigmentation, the absence of lymphatic networks and stable vascular, hair follicles, sweat glands, adipose, and inadequate innervation are some of the missing elements of cellular or biological skin replacements equation [7].', 'text_sha256': 'b131d19b1511a27c6707c7bdda75f87c5d5d0e51a09ff50a949d5509d1c8d4a2'},
{'text': 'The complete restoration of the anatomy and physiology of uninjured skin is highly dependent on the modulation of biological pathways of embryonic and fetal formation.', 'text_sha256': '89d412d0aa5c1ef5b7e39ffbe53997f256583e668e167f9e7bff52754c1eaba6'},
{'text': 'developmental biology into possible biological skin replacements can have a full recovery of anatomy and physiology, along with eliminating morbidity from skin wounds In recent years, improvements in early wound excision, respiratory assistance, fluid revival, treatment of inhalation damage, enhanced immune function, infection prevention, integration of aerobic activity during healing, and advancement of anti-scarring techniques have all been accomplished in burn care.', 'text_sha256': 'b3813216374fb4ba0169d73ea0706708f7885998f6152f67f82ec4a4f5f2f91f'},
{'text': 'These advancements also resulted in substantial decreases in death, hospitalization, and long-term morbidity.', 'text_sha256': '4356f0a81812d5e028f56ecebb41ea5efcb776ae840a4c6b67de14a456a1174e'},
{'text': 'Many strategies for healing deep partial and full-thickness burns have been developed by researchers, including the placement of the epidermal barrier through autologous or allogeneic transplantation.', 'text_sha256': '707dd6da263e663d9ac7d5657cf808f864904c42501ddf9f9cb7280a70229976'},
{'text': 'Autologous cells can permanently reside on the wound to have definitive wound closure.', 'text_sha256': '010afc176b1bfd8be2dea31503c53b185c45dff6c668d8ae0ee49c8d1af5bbdd'},
{'text': 'In contrast, allogeneic cells can stay on the wound for a few days or weeks; However, the combination of the allogeneic fibroblast with biodegradable scaffolds would provide extracellular matrix and growth factors components would facilitates faster wound closure by motivating autologous healing, but it does not last long maybe for only a few days to weeks.', 'text_sha256': '0139fe4a6ab7b550a466f51d1699f61239685e836b74bc87713b825b06c599d9'},
{'text': 'Although these methods do tend to decrease mortality in major burns, after transplantation, they neglect hair follicles and glands.', 'text_sha256': '7a5746acd82a291f691a520b28f98ff131217c2aac56eb78f2b2290ab067ccde'},
{'text': 'Scientists use techniques which can damage cells such as multi-layering of a variety of cells and the transition of cell-matrix droplets onto a substrate known as "ink-jet printing," by amplifying a laser pulse.', 'text_sha256': 'f7126a142a7bb7ddc4530773ee50c5c453fc4c2c376dab93ce281ba0956b7e22'},
{'text': 'The brief exposures to high pressure, temperature, or biochemical toxins would damage the cells [7].', 'text_sha256': 'eaeaadcde60d27b46b7429de0e21deaa53a89f37044f8dc7d927234edb9f7bad'},
{'text': 'autologous engineered skin substitutes.', 'text_sha256': '8fb92691db51d527740cbd0e37266048ebbbbbdce8815defe98d52b4aa82d058'},
{'text': 'Regenerative medicine is meant to restore the functionality of a harmed,   malfunctioning, or absent tissue.', 'text_sha256': '252852d6d5d13fb6e1325406b0f6923403a5bcd729e685dafdfc662d2637eb2a'},
{'text': 'In regenerative medicine, there are three primary methods.', 'text_sha256': '05819b82607e2021c8ffc804b524f2a0fff3385425d6e32b0f37d302627a3b70'},
{'text': 'Cell-based therapies are the first approach executed by the cells that are delivered to either directly regenerate a tissue or through the paracrine functions.', 'text_sha256': '8f48ec7f62b61d191d6d02252f2e9a6bc00698e589d2b48b0022f6b28088befc'},
{'text': 'The second method, the material-based methods focuses on biodegradable materials that are often fabricated with cellular functions.', 'text_sha256': 'd62414f38d3f59b55e7c749e9fb683915c37f246c0a0182abe6e2085ed1dc486'},
{'text': 'The most important method, that is the main focus of this paper, is tissue engineering: combining the use of cells and a biodegradable scaffold to construct a tissue [4].', 'text_sha256': '8d27bc918be4d8e1333083754c629628eb80a6316b92810acfb94117370b280d'},
{'text': 'At the base of hair follicle cells ADSCs (adipose tissue-related stem cells) are mesenchymal stem cells found in subcutaneous tissue.', 'text_sha256': '7968a81b8f6a796d157d5a045f9b4bc51fca3e9d9004aec1e66bab81666277d4'},
{'text': 'Controlling skin regeneration, aging, and structural deficits are significant roles of these cells [8].', 'text_sha256': 'cfacb0de086c5fccf399edbfe922fefc4cb0ef65c24d0835b565f4c30efc68a2'},
{'text': 'Proliferating and dividing into skin cells to replace damaged or dead cells are known to be done by ADSCs.', 'text_sha256': 'eae84a46995889878cf1938fb3027fc57f6aa2b8438dc4af8e0626a150f74402'},
{'text': 'Additionally, they also stimulate cell replication and the healing process through an autocrine and paracrine pathway by stimulating CD44 fibroblast receptors.', 'text_sha256': '6d1b5f57b7c211fb642db9b0e21d6e5bf9f61bff92f78e0683c128f045c50569'},
{'text': 'During wound recovery, ADSCs have a high capacity to migrate to wounded sites and are recruited quickly, in addition to their division into dermal fibroblasts (DF), endothelial cells, and keratinocytes.', 'text_sha256': 'da18fbf792720a037844102d4d6a16c16f628e0b5c38b8db8c948c4670fa3200'},
{'text': 'Adipose-derived stem cells are primarily more proliferative and have immunoregulatory properties, which means they are isolated in a less disruptive and more reproducible manner with the capability of inactivating T cells [8].', 'text_sha256': '031e9cbf2f2583efa10c22db84433800b4d955ee17a9a1f8dac6c20001ea8588'},
{'text': 'Compared to bone marrow, ADSCs have higher multipotency, which is actually needed more for ectodermal and endodermal tissue repair.', 'text_sha256': 'c4356d972388f0120f31570a98fcdfb46a1820d0d138d268816603e5b3aac164'},
{'text': 'ADSCs will secrete a diverse secretome, resulting in increased cell proliferation and differentiation, migration, and enhanced cellular and microenvironment safety.', 'text_sha256': '1cc2a47c1aa5733bf62513936f17bc6d4e446d045008a120bb0f99a93cc2385d'},
{'text': 'A set of trophic factors that secretome correlates to cytokines, growth factors, and chemokines that enable ADSCs to function as paracrine tools rather than cell ADSCs were also discovered in subcutaneous tissue.', 'text_sha256': '41a24c5819d69ef35bf9650bc5c3892d2668b06249ca4e7d2c8afb762534c9bf'},
{'text': 'This involvement plays an essential role in skin restoration and regeneration by retaining and rejuvenating skin tissue structure (even it can be as a physiological reaction to local injury) by seeding younger cells to the epidermis, which they later differentiate into dermal fibroblasts, keratinocytes, and other The tendency of different epithelial cells to secrete growth factors and extracellular matrix (ECM) proteins maintains a differentiation of rejuvenation and regeneration in the ADSC microenvironment.', 'text_sha256': '8efc3f9874fb8a5785e26c2db7bc0c33c68e728c74a4e902154349766fac46ce'},
{'text': 'Furthermore, the primary sources of extracellular matrix (ECM) are ADSCs and DFs proteins involved in skin structure and functionality.', 'text_sha256': '6ea83e4f6f61370bacd568f4937a18a6f22a09b0ac242dcda65e9f9ab5871b6b'},
{'text': 'Extracellular vesicles are lipid bilayer vesicles that include several RNA organisms (including mRNAs and miRNAs), soluble (cytosolic) proteins, and transmembrane proteins in the proper and functional orientation [8].', 'text_sha256': 'b419be85e36b6dfcaea9fc203f62b7d6a5ea2d6b5d4748456f33e74518ed2eb5'},
{'text': 'EVs are involved in various mechanisms, membrane protein, lipid recycling, immune regulation, apoptosis, angiogenesis, cellular proliferation, differentiation, and tissue engineering to modulate cell recruitment.', 'text_sha256': '659f82f4c60d78ff4c268141fe8baa20faa6a3f18d5190910a391133ab3f4a94'},
{'text': 'Exosomes, which are emitted through the fusion of the multivesicular structure with both microvesicles and the cell membrane that have been released directly from the cell membrane, are examples of extracellular vesicles (EV) [4].', 'text_sha256': 'b4232482009734fa050ce23a6f943fcc1fbf0c14d7b33dad6024f4d840c194d6'},
{'text': 'The many processes in which EVs are active and the elasticity in which they may affect the activity of recipient cells allows EV, a fascinating subject, to be utilized for both therapeutic and diagnostic purposes.', 'text_sha256': '37011fab65168b949e30d50e790c83509018c12f08db33276b8578040397b90b'},
{'text': 'EVs are implicated in tissue and organ injury reconstruction and can clarify any paracrine symptoms seen in stem cell-based therapeutic approaches.', 'text_sha256': '0eacef6843720499a02bc7df7a6c18d6d74c07c4a4a525f5c34a91b193ea8ef4'},
{'text': 'MSCs (multipotent mesenchymal stem cells) are discovered as residual stem cells in majority adult organs, especially adipose tissue.', 'text_sha256': '596a0a6409088cec66898737dc1ef86639c528d49c101db650d84629bea30b23'},
{'text': 'These cells exhibit standard mesenchymal cell properties in vitro and are segregated from the mesenchymal vascular fraction.', 'text_sha256': '35eebfbed314ecbc231c5fd21c1e983c8639dfd2daa3a209fe6899f2d1afad72'},
{'text': 'Extracellular vesicles have a paracrine impact on cell morphology, migration, proliferation, and differentiation.', 'text_sha256': 'a6780f3e9a19d5e98c5780430515aa6e6292e551b2a7a1cbf97553a2d416283e'},
{'text': 'These paracrine results of EV can be helpful in regenerative medicine.', 'text_sha256': '7b6cf3da01fab34c273634c2ed46ddf107a9aac40f3aea8dd47b00122f02e98d'},
{'text': 'EV may be used in regenerative therapy in various ways, like injection, combining with hydrogels, and covering scaffolds with EV utilizing fibrous tissue hydrogel.', 'text_sha256': '00e182fc2a8ab26ae4aff43a530847dca0a3c3198496bb23ace2d968947f9f98'},
{'text': 'Intercellular communications are essential roles of EVs through secretome that engage in.', 'text_sha256': '020c33647684a1f52b8201ba14c6134c983318b0bce099fe3002149f1e79fda3'},
{'text': 'Thus, their content and functionality can vary based on the conditions of the vesicular generating cells.', 'text_sha256': '464608b58f65d3a5eca455f5948a30b6d131a0b85dae1a331633c7584aac507d'},
{'text': 'Changes in EV content in response to conditions specific to growth would play a crucial role in these cases by affecting tissue regeneration and wound repair by identifying new pathways into Epidermolysis bullosa is a genodermatosis, a hereditary skin disease that usually mutations of genes encoding extracellular matrix proteins (collagen) are the primary cause.', 'text_sha256': '6bc42056308ecd2168a0bc29496b40e166832a09bb07efe85f9de69b330fe4ba'},
{'text': 'Severe and prolonged skin blistering with extreme and potentially fatal complications are examples of this disorder.', 'text_sha256': '6c399b612da5b7ec4ed737829f7fc405b25ec3ba4281643fd6243d662cd561b3'},
{'text': 'There is no treatment for epidermolysis bullosa at the moment.', 'text_sha256': '44b825cc6a687aa5a22c503995c5609014493ebefdebdaf3f61ae815fd674ecd'},
{'text': 'However, enabling autologous therapies developed from a patient-specific renewable community of cells count as advantages in gene therapy and editing methods, along with the development of pluripotent stem cells from patients with epidermolysis Another example of skin disease is diabetic foot ulcers, known as (DFU), which is a significant difficulty that often results in infection or even death.', 'text_sha256': 'c809338ff4eecce1e5831fec6cb2346fc98c85befcd29b09ec4cffd2002173a5'},
{'text': 'The mesenchymal stem cell-based therapeutic method has shown effectiveness in wound treatment, presenting a theoretically systematic alternative by treating various microaerobic Base Material and The Interaction Damaged tissue repair entails the existence of cells capable of repairing the damaged tissue and the presence of a microenvironment that facilitates adequate tissue regeneration.', 'text_sha256': 'c809024aa7cf38135ee44ba4c6551890759bd1ce691778bb12ae99cb3bf9c809'},
{'text': 'Furthermore, cells must be directed to form a structure of the suitable size, shape, and in certain situations, structural support.', 'text_sha256': 'dba170f904e8761a8a62fbeb41a6cf6f23a149a8b336d989d66d746fdc884b80'},
{'text': 'In damaged tissue, the extracellular metric is often damaged, functionally disabled, or missing, while In healthy tissue, the ECM guides and Scaffolds are structures that include the conditions for cell retention and tissue regeneration and are used to solve this problem.', 'text_sha256': '64e76a6b4edec76b32a1c8516176f54c5f286f83a48fe903ad5f3625e395fd6f'},
{'text': 'Scaffolds may be either natural (decellularized ECM or enhanced elastin - or collagen gels) or synthetic (synthetic hydrogels or porous polymer scaffolds) [4].', 'text_sha256': 'a19fb3bf7aebb6a71b9b57786d6ecc8123c94e6b8281c6c2c6cc920026c83890'},
{'text': 'To enable suitable cell populations, artificial porous scaffolds must meet precise criteria.', 'text_sha256': '875216d51e6aad7527563115d0d495a316b14b92261569b121d86c923ee936f7'},
{'text': 'A synthetic scaffold, ideally, offers biodegradable, and is gradually substituted by autologous ECM.', 'text_sha256': 'a3e485f6c727230814151e5889df52594115dc7e644f5e90ecf0f9f0c2820efc'},
{'text': 'By providing a porous structure required, cells would be able to move or they can be seeded in the scaffold to provide an environment with a sufficient supply of nutrients [4].', 'text_sha256': '9d98bd7dfb8df230f236e9423f1259fe0ca65c7f48e77f1d0b3f50995af2e480'},
{'text': 'Cellular components are removed from the tissue using a mixture of enzymatic, physical, and chemical therapies to reduce the possibility of immune responses against antigens and the spread of pathogens in these scaffolds.', 'text_sha256': '834c2b2a36b95f5dc66679f7878b153e0237e36eba39577782dc02b3a1fe355d'},
{'text': 'Electrospinning is the most widely utilized technique for creating porous synthetic scaffolds since it allows for the development of structures of complicated geometry variations of fiber forms in both mixed and layered configurations.', 'text_sha256': '121db6b4b3457715f000c989e22aec5b4b5c04955e3726f9c28f1ab91c421930'},
{'text': 'Polycaprolactone (PCL), polyglycolic acid (PGA), polyhydroxyalkanoate (PHA), and polylactic acid (PLA) are biodegradable polymers used in electrospinning.', 'text_sha256': 'd15ee5ffbda675bba81a95b0d1c76c6abd7406e6210a6b1fe3e7bc62ed78eb20'},
{'text': 'Combining fiber types in complex patterns, scaffold degradability, durability, and biological activity can be modulated.', 'text_sha256': 'd58f64b34002f3e3412e9f149780cf0d5b2bffe365c37389a8e5ecb8ce1a86df'},
{'text': 'MSC migration, proliferation, and osteogenic differentiation are improved by coating the scaffold with collagen peptide as an example.', 'text_sha256': 'd980a6af0e9d3a4484ce332f2800783c1a7b7301df5142e2cf600c57738b4344'},
{'text': 'Scaffolds may also be engineered to release proteins, peptides, or cytokines through degradation or coating fibers with a biodegradable material like fibrin or gelatin containing a combination of these bioactive molecules.', 'text_sha256': '5bf72a65a50ac314bd12fa554cf8fced84a15f2997f2ee83d8ae84ff8e864714'},
{'text': 'Also, because of their heavily hydrated structure, low toxicity, and proximity to the extracellular matrix, hydrogels are the most commonly used bio-inks in tissue engineering to build scaffolds.', 'text_sha256': '0d5d7bce7206e13032d0a92447061f6cb914ca816ae5164fff7cc7ae1d0862cc'},
{'text': 'As a result, bio-ink built with gelatin and natural polymers is compatible with various soft tissue cell types and can be combined with biological molecules such as proteins, growth factors, and     Many people suffering from skin disorders such as chronic wounds, non-healing ulcers, and diabetic ulcers need skin repair and regeneration.', 'text_sha256': 'f927f3608fe0be3b3fad54720b8b364a28be8a452c0b076e8132f26b615eedee'},
{'text': 'Aside from the diseases listed above, the industry needed a skin rejuvenation system and regeneration for cosmetic purposes.', 'text_sha256': 'eb86f6f003005424184ed01388dbd71346362cc34061fa9f759d78d72defb74d'},
{'text': 'The procedure used to deliver pluripotent stem cells to the desired tissue was known as reconstructive medicine.', 'text_sha256': '565b08df3338c281e90b4250aa1a49af8a62266a63af81ff76a2cf9ef8550f17'},
{'text': 'Mesenchymal stem cells are the most fascinating since, when put in the correct setting and stimulated with the proper growth factors, they could choose a pathway to differentiate into the desired tissue.', 'text_sha256': '08440e55b47bb9991b3bc21728f01aa8b30a91c5640ffb0297dec41c426a56d0'},
{'text': 'They are also very available, inexpensive, simple to extract, and reproducible.', 'text_sha256': '293f7e0b57ad326235531d57de1e0a1091db445396dd3955c48ced1b426feb0a'},
{'text': 'These mesenchymal stem cells are derived from bone marrow, bone, connective tissues, fats, and other tissues.', 'text_sha256': '81793a161e6547790e7df493bf0d38c259acb239d270e419dd20b646c3157807'},
{'text': 'Fat is the ideal reservoir for mesenchymal cells that have recently been identified.', 'text_sha256': '19c07c64ccafeb2531d7eae3f1806070ec5a6b336e8c9397c3653bb7ecae2f82'},
{'text': 'Fibrous, collagen fibers and fibroblasts On one side, by injecting collagenase enzymes into the fat, all of the collagen fibers and fibroblasts can be removed, leaving just the pure mesenchymal stem cells.', 'text_sha256': '31a6ae4fd46177618fe67811fdf6d56060909bac378568cab98d6ec696f948dd'},
{'text': 'In the engineered stem cell vascular fraction (SVF), a pellet comprising mesenchymal cells, growth factors, inflammatory cells such as macrophage-monocyte, and fibroblasts, the mesenchymal cells are segregated from the other components through centrifugation.', 'text_sha256': '9164dc8d58cd90285401d302a03e671dd668150307ae27f85345bbd411713c2c'},
{'text': 'Placing mesenchymal stem cells in the target environment and supplying the required growth factors can result in the formation of fibroblasts, the production of collagen, and the reconstruction of the skin.', 'text_sha256': '6b9018fea8ca9fd50254e7e499d64c3ac115704f29bacbc566fb093fcf0284e1'},
{'text': 'As previously stated, the stem cell condition is essential, as there is a risk of movement of the stem cell due to blood circulation and paracrine, which the stem cell will secrete and influence its environment.', 'text_sha256': '6f41867f3f6f6b038edb081b98fc2f9b03a6cb480d16fc3320d924ba44a7014f'},
{'text': 'As a result, the scaffold is the structure that prevents the stem cell from being flushed away by the blood supply and disrupting the environment.', 'text_sha256': 'fd00c31d43d1623e58642e38511ba129e63f327aa2a41b968fc948decdad4a51'},
{'text': 'The scaffold must have unique properties that prevent foreign body reactions, are not antigenic or allergic, and   do not induce infection.', 'text_sha256': 'ae7ea0e878ea166d17ab03dfd80fb4ba4f8d0a6c44e5d0a29d4e3f1cd3b404a8'},
{'text': 'It must also be affordable and readily available.', 'text_sha256': '0e37067b910efd4e7a6ea4396f75d08751ea71114770898cce118b8e9eed3be9'},
{'text': 'Alginate scaffolds have been used in the most recent process.', 'text_sha256': '0ca88dcc99aab9f89ed2403003aa1af1469b1e9f1ca34a6e18eca368414e809b'},
{'text': "The scaffold containing the mesenchymal and critical growth factors into the desired cell will, for example, activate the CD44 receptors on the skin's fibroblasts, increasing collagen output.", 'text_sha256': 'f3d6fc91d52f395e639325b188564d7f20cb41edf31cc080ff1d4bff927976dc'},
]

In [ ]:
pip install neo4j

  Using cached neo4j-5.15.0.tar.gz (196 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.15.0-py3-none-any.whl size=272484 sha256=ce6173f02e9c724359f2c3c9218dccc0fb60d4ba99565e4f69260cfa55ada59a
  Stored in directory: /root/.cache/pip/wheels/d5/08/10/6371dbdeec2efd7782f559b21c32bb6b4192ae0216ec5e39c5
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://18.234.240.27:7687'
user = 'neo4j'
password = 'catch-dockings-sight'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
author = article_txt.split("\n")[0]
title = " ".join(article_txt.split("\n")[2:4])

neo4j_query("""
MERGE (a:Author{name:$author})
MERGE (b:Article{title:$title})
MERGE (a)-[:WROTE]->(b)
""", {'title':title, 'author':author})

""


In [ ]:
neo4j_query("""
MATCH (a:Article)
UNWIND $data as row
MERGE (s:Sentence{id:row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s, row.entities as entities
UNWIND entities as entity
MERGE (e:Entity{id:entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
              e.name = entity.entity,
              e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
""", {'data': parsed_entities})

""


#Search Engine

In [ ]:
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "autoimmune diseases"
RETURN s.text as result
""")

#Co-occurrence analysis

In [ ]:
neo4j_query("""
MATCH (e1:Entity)<-[:MENTIONS]-()-[:MENTIONS]->(e2:Entity)
WHERE id(e1) < id(e2)
RETURN e1.name as entity1, e2.name as entity2, count(*) as cooccurrence
ORDER BY cooccurrence
DESC LIMIT 3
""")

# Inspect author expertise


In [ ]:
neo4j_query("""
MATCH (a:Author)-[:WROTE]->()-[:HAS_SENTENCE]->()-[:MENTIONS]->(e:Entity)
RETURN a.name as author, e.name as entity, count(*) as count
ORDER BY count DESC
LIMIT 5
""")